# Lab | Tools prompting

**Replace the existing two tools decorators, by creating 3 new ones and adjust the prompts accordingly**

### How to add ad-hoc tool calling capability to LLMs and Chat Models

:::{.callout-caution}

Some models have been fine-tuned for tool calling and provide a dedicated API for tool calling. Generally, such models are better at tool calling than non-fine-tuned models, and are recommended for use cases that require tool calling. Please see the [how to use a chat model to call tools](https://python.langchain.com/docs/how_to/tool_calling/) guide for more information.

In this guide, we'll see how to add **ad-hoc** tool calling support to a chat model. This is an alternative method to invoke tools if you're using a model that does not natively support tool calling.

We'll do this by simply writing a prompt that will get the model to invoke the appropriate tools. Here's a diagram of the logic:

<br>

![chain](https://education-team-2020.s3.eu-west-1.amazonaws.com/ai-eng/tool_chain.svg)

## Setup

We'll need to install the following packages:

In [1]:
pip install --upgrade --quiet langchain langchain-community langchain_openai

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-experimental 0.3.4 requires langchain-community<0.4.0,>=0.3.0, but you have langchain-community 0.4.1 which is incompatible.
langchain-experimental 0.3.4 requires langchain-core<0.4.0,>=0.3.28, but you have langchain-core 1.2.7 which is incompatible.


If you'd like to use LangSmith, uncomment the below:

In [11]:
import getpass
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

You can select any of the given models for this how-to guide. Keep in mind that most of these models already [support native tool calling](https://python.langchain.com/docs/integrations/chat), so using the prompting strategy shown here doesn't make sense for these models, and instead you should follow the [how to use a chat model to call tools](https://python.langchain.com/docs/how_to/tool_calling/) guide.

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs openaiParams={`model="gpt-4"`} />
```

To illustrate the idea, we'll use `phi3` via Ollama, which does **NOT** have native support for tool calling. If you'd like to use `Ollama` as well follow [these instructions](https://python.langchain.com/docs/integrations/chat/ollama).

In [12]:
from langchain_community.llms import Ollama

model = Ollama(model="phi3")

In [1]:
from langchain_openai import ChatOpenAI
import os

# Configura tu API key de OpenAI
os.environ["OPENAI_API_KEY"] = ''

# Usa ChatOpenAI en lugar de Ollama
model = ChatOpenAI(model="gpt-4", temperature=0)

c:\Users\Mauricio\miniconda3\envs\'v2025'\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



#  How to Install and Run Ollama with the Phi-3 Model

This guide walks you through installing **Ollama** and running the **Phi-3** model on Windows, macOS, and Linux.

---

## Windows

1. **Download Ollama for Windows**  
   Go to: [https://ollama.com/download](https://ollama.com/download)  
   Download and run the installer.

2. **Verify Installation**  
   Open **Command Prompt** and type:
   ```bash
   ollama --version
   ```

3. **Run the Phi-3 Model**  
   In the same terminal:
   ```bash
   ollama run phi3
   ```

4. **If you get a CUDA error (GPU memory issue)**  
   Run Ollama in **CPU mode**:
   ```bash
   set OLLAMA_NO_CUDA=1
   ollama run phi3
   ```

---

##  macOS

1. **Install via Homebrew**  
   Open the Terminal and run:
   ```bash
   brew install ollama
   ```

2. **Run the Phi-3 Model**
   ```bash
   ollama run phi3
   ```

3. **To force CPU mode (no GPU)**
   ```bash
   export OLLAMA_NO_CUDA=1
   ollama run phi3
   ```

---

##  Linux

1. **Install Ollama**  
   Open a terminal and run:
   ```bash
   curl -fsSL https://ollama.com/install.sh | sh
   ```

2. **Run the Phi-3 Model**
   ```bash
   ollama run phi3
   ```

3. **To force CPU mode (no GPU)**
   ```bash
   export OLLAMA_NO_CUDA=1
   ollama run phi3
   ```

---

##  Notes

- The first time you run `ollama run phi3`, it will **download the model**, so make sure you’re connected to the internet.
- Once downloaded, it works **offline**.
- Keep the terminal open and running in the background while using Ollama from your code or notebook.


## Create a tool

First, let's create an `add` and `multiply` tools. For more information on creating custom tools, please see [this guide](https://python.langchain.com/docs/how_to/custom_tools/).

In [3]:
from langchain_core.tools import tool


@tool
def multiply(x: float, y: float) -> float:
    """Multiply two numbers together."""
    return x * y


@tool
def add(x: int, y: int) -> int:
    "Add two numbers."
    return x + y


tools = [multiply, add]

# Let's inspect the tools
for t in tools:
    print("--")
    print(t.name)
    print(t.description)
    print(t.args)

--
multiply
Multiply two numbers together.
{'x': {'title': 'X', 'type': 'number'}, 'y': {'title': 'Y', 'type': 'number'}}
--
add
Add two numbers.
{'x': {'title': 'X', 'type': 'integer'}, 'y': {'title': 'Y', 'type': 'integer'}}


In [4]:
multiply.invoke({"x": 4, "y": 5})

20.0

## Creating our prompt

We'll want to write a prompt that specifies the tools the model has access to, the arguments to those tools, and the desired output format of the model. In this case we'll instruct it to output a JSON blob of the form `{"name": "...", "arguments": {...}}`.

In [5]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
print(rendered_tools)

multiply(x: float, y: float) -> float - Multiply two numbers together.
add(x: int, y: int) -> int - Add two numbers.


In [6]:
system_prompt = f"""\
You are an assistant that has access to the following set of tools. 
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. 
Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with keys corresponding 
to the argument names and the values corresponding to the requested values.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [7]:
chain = prompt | model
message = chain.invoke({"input": "what's 3 plus 1132"})

# Let's take a look at the output from the model
# if the model is an LLM (not a chat model), the output will be a string.
if isinstance(message, str):
    print(message)
else:  # Otherwise it's a chat model
    print(message.content)

{
  "name": "add",
  "arguments": {
    "x": 3,
    "y": 1132
  }
}


## Adding an output parser

We'll use the `JsonOutputParser` for parsing our models output to JSON.

In [8]:
from langchain_core.output_parsers import JsonOutputParser

chain = prompt | model | JsonOutputParser()
chain.invoke({"input": "what's thirteen times 4"})

{'name': 'multiply', 'arguments': {'x': 13, 'y': 4}}

:::{.callout-important}

🎉 Amazing! 🎉 We now instructed our model on how to **request** that a tool be invoked.

Now, let's create some logic to actually run the tool!
:::

## Invoking the tool 🏃

Now that the model can request that a tool be invoked, we need to write a function that can actually invoke 
the tool.

The function will select the appropriate tool by name, and pass to it the arguments chosen by the model.

In [9]:
from typing import Any, Dict, Optional, TypedDict

from langchain_core.runnables import RunnableConfig


class ToolCallRequest(TypedDict):
    """A typed dict that shows the inputs into the invoke_tool function."""

    name: str
    arguments: Dict[str, Any]


def invoke_tool(
    tool_call_request: ToolCallRequest, config: Optional[RunnableConfig] = None
):
    """A function that we can use the perform a tool invocation.

    Args:
        tool_call_request: a dict that contains the keys name and arguments.
            The name must match the name of a tool that exists.
            The arguments are the arguments to that tool.
        config: This is configuration information that LangChain uses that contains
            things like callbacks, metadata, etc.See LCEL documentation about RunnableConfig.

    Returns:
        output from the requested tool
    """
    tool_name_to_tool = {tool.name: tool for tool in tools}
    name = tool_call_request["name"]
    requested_tool = tool_name_to_tool[name]
    return requested_tool.invoke(tool_call_request["arguments"], config=config)

Let's test this out 🧪!

In [10]:
invoke_tool({"name": "multiply", "arguments": {"x": 3, "y": 5}})

15.0

## Let's put it together

Let's put it together into a chain that creates a calculator with add and multiplication capabilities.

In [11]:
chain = prompt | model | JsonOutputParser() | invoke_tool
chain.invoke({"input": "what's thirteen times 4.14137281"})

53.83784653

## Returning tool inputs

It can be helpful to return not only tool outputs but also tool inputs. We can easily do this with LCEL by `RunnablePassthrough.assign`-ing the tool output. This will take whatever the input is to the RunnablePassrthrough components (assumed to be a dictionary) and add a key to it while still passing through everything that's currently in the input:

In [12]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output=invoke_tool)
)
chain.invoke({"input": "what's thirteen times 4.14137281"})

{'name': 'multiply',
 'arguments': {'x': 13, 'y': 4.14137281},
 'output': 53.83784653}

## What's next?

This how-to guide shows the "happy path" when the model correctly outputs all the required tool information.

In reality, if you're using more complex tools, you will start encountering errors from the model, especially for models that have not been fine tuned for tool calling and for less capable models.

You will need to be prepared to add strategies to improve the output from the model; e.g.,

1. Provide few shot examples.
2. Add error handling (e.g., catch the exception and feed it back to the LLM to ask it to correct its previous output).

In [14]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = f"""\
You are an assistant that has access to the following set of tools. 
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. 
Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with keys corresponding 
to the argument names and the values corresponding to the requested values.

Examples:
User: What is 5 times 3?
Assistant: {{"name": "multiply", "arguments": {{"x": 5, "y": 3}}}}

User: Add 10 and 20
Assistant: {{"name": "add", "arguments": {{"x": 10, "y": 20}}}}

User: Calculate 7.5 multiplied by 2.3
Assistant: {{"name": "multiply", "arguments": {{"x": 7.5, "y": 2.3}}}}
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [15]:
from typing import Any, Dict, Optional, TypedDict
from langchain_core.runnables import RunnableConfig
import json


def invoke_tool_with_error_handling(
    tool_call_request: ToolCallRequest, 
    config: Optional[RunnableConfig] = None,
    max_retries: int = 3
):
    """
    Invoca una herramienta con manejo de errores.
    
    Args:
        tool_call_request: Dict con 'name' y 'arguments'
        config: Configuración de LangChain
        max_retries: Número máximo de reintentos
        
    Returns:
        Resultado de la herramienta o mensaje de error
    """
    tool_name_to_tool = {tool.name: tool for tool in tools}
    
    try:
        # Validar que el nombre de la herramienta existe
        name = tool_call_request.get("name")
        if not name:
            return {"error": "No tool name provided", "request": tool_call_request}
        
        if name not in tool_name_to_tool:
            available_tools = ", ".join(tool_name_to_tool.keys())
            return {
                "error": f"Tool '{name}' not found. Available tools: {available_tools}",
                "request": tool_call_request
            }
        
        # Validar que los argumentos existen
        arguments = tool_call_request.get("arguments")
        if not arguments:
            return {"error": "No arguments provided", "request": tool_call_request}
        
        # Invocar la herramienta
        requested_tool = tool_name_to_tool[name]
        result = requested_tool.invoke(arguments, config=config)
        
        return {"success": True, "result": result, "tool": name, "arguments": arguments}
        
    except KeyError as e:
        return {
            "error": f"Missing required argument: {str(e)}",
            "request": tool_call_request
        }
    except TypeError as e:
        return {
            "error": f"Invalid argument type: {str(e)}",
            "request": tool_call_request
        }
    except Exception as e:
        return {
            "error": f"Unexpected error: {str(e)}",
            "request": tool_call_request
        }

In [5]:
from langchain_core.tools import tool


@tool
def multiply(x: float, y: float) -> float:
    """Multiply two numbers together."""
    return x * y


@tool
def add(x: int, y: int) -> int:
    "Add two numbers."
    return x + y


tools = [multiply, add]

In [10]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description
import json


def safe_parse_json(output):
    """Parsea JSON de forma segura"""
    try:
        if isinstance(output, str):
            return json.loads(output)
        elif hasattr(output, 'content'):
            return json.loads(output.content)
        return output
    except json.JSONDecodeError as e:
        return {
            "error": f"Invalid JSON format: {str(e)}",
            "raw_output": str(output)
        }


def execute_chain_with_retry(user_input: str, max_attempts: int = 3):
    """
    Ejecuta la cadena con reintentos si hay errores.
    
    Args:
        user_input: Pregunta del usuario
        max_attempts: Número máximo de intentos
        
    Returns:
        Resultado final o error
    """
    # Store original input
    original_input = user_input
    
    # Render tools inside the function
    rendered_tools = render_text_description(tools)
    
    # Define the prompt here with the rendered_tools
    system_prompt = f"""\
You are an assistant that has access to the following set of tools. 
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. 
Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with keys corresponding 
to the argument names and the values corresponding to the requested values.

Examples:
User: What is 5 times 3?
Assistant: {{"name": "multiply", "arguments": {{"x": 5, "y": 3}}}}

User: Add 10 and 20
Assistant: {{"name": "add", "arguments": {{"x": 10, "y": 20}}}}

User: Calculate 7.5 multiplied by 2.3
Assistant: {{"name": "multiply", "arguments": {{"x": 7.5, "y": 2.3}}}}
"""

    prompt = ChatPromptTemplate.from_messages(
        [("system", system_prompt), ("user", "{input}")]
    )
    
    chain = prompt | model
    
    for attempt in range(max_attempts):
        print(f"\n--- Attempt {attempt + 1}/{max_attempts} ---")
        
        try:
            # Obtener respuesta del modelo
            message = chain.invoke({"input": original_input})
            
            # Extraer contenido
            if isinstance(message, str):
                content = message
            else:
                content = message.content
            
            print(f"Model output: {content}")
            
            # Parsear JSON
            parsed = safe_parse_json(content)
            
            if "error" in parsed:
                print(f"Parse error: {parsed['error']}")
                if attempt < max_attempts - 1:
                    continue
                return parsed
            
            # Invocar herramienta
            result = invoke_tool_with_error_handling(parsed)
            
            if "error" in result:
                print(f"Tool error: {result['error']}")
                if attempt < max_attempts - 1:
                    continue
            
            return result
            
        except Exception as e:
            print(f"Unexpected error: {str(e)}")
            if attempt < max_attempts - 1:
                continue
            return {"error": f"Failed after {max_attempts} attempts: {str(e)}"}
    
    return {"error": f"Failed after {max_attempts} attempts"}


# Probar
result = execute_chain_with_retry("what's 13 times 4.5")
print("\n--- Final Result ---")
print(result)


--- Attempt 1/3 ---
Unexpected error: 'Input to ChatPromptTemplate is missing variables {\'"name"\'}.  Expected: [\'"name"\', \'input\'] Received: [\'input\']\nNote: if you intended {"name"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{"name"}}\'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT '

--- Attempt 2/3 ---
Unexpected error: 'Input to ChatPromptTemplate is missing variables {\'"name"\'}.  Expected: [\'"name"\', \'input\'] Received: [\'input\']\nNote: if you intended {"name"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{"name"}}\'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT '

--- Attempt 3/3 ---
Unexpected error: 'Input to ChatPromptTemplate is missing variables {\'"name"\'}.  Expected: [\'"name"\', \'input\'] Received: [\'input\']\nNote: if you intend

In [7]:
# Probar con diferentes casos
test_cases = [
    "what's 5 plus 10",
    "multiply 7 by 3",
    "what's 15.5 times 2.2",
    "invalid query xyz"  # Este debería fallar gracefully
]

for test in test_cases:
    print(f"\n{'='*50}")
    print(f"Testing: {test}")
    result = execute_chain_with_retry(test)
    print(f"Result: {result}")


Testing: what's 5 plus 10

--- Attempt 1/3 ---
Unexpected error: 'Input to ChatPromptTemplate is missing variables {\'"name"\'}.  Expected: [\'"name"\', \'input\'] Received: [\'input\']\nNote: if you intended {"name"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{"name"}}\'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT '

--- Attempt 2/3 ---
Unexpected error: 'Input to ChatPromptTemplate is missing variables {\'"name"\'}.  Expected: [\'"name"\', \'input\'] Received: [\'input\']\nNote: if you intended {"name"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{"name"}}\'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT '

--- Attempt 3/3 ---
Unexpected error: 'Input to ChatPromptTemplate is missing variables {\'"name"\'}.  Expected: [\'"name"\', \'input\'] Received: [\'in